# 3201. Find the Maximum Length of Valid Subsequence I

**Medium**

You are given an integer array nums.
A subsequence sub of nums with length x is called valid if it satisfies:

> (sub[0] + sub[1]) % 2 == (sub[1] + sub[2]) % 2 == ... == (sub[x - 2] + sub[x - 1]) % 2.

> Return the length of the longest valid subsequence of nums.

A subsequence is an array that can be derived from another array by deleting some or no elements without changing the order of the remaining elements.

Example 1:

```python
Input: nums = [1,2,3,4]

Output: 4
```

**Explanation:**

> The longest valid subsequence is [1, 2, 3, 4].

# Example 2:

```python
Input: nums = [1,2,1,1,2,1,2]

Output: 6
```

**Explanation:**

> The longest valid subsequence is [1, 2, 1, 2, 1, 2].

# Example 3:

```python
Input: nums = [1,3]

Output: 2
```

**Explanation:**

> The longest valid subsequence is [1, 3].

**Constraints:**

- 2 <= nums.length <= 2 \* 105
- 1 <= nums[i] <= 107


In [ ]:
# Approach 1: Counting (Optimal and Simplest for k=2)

class Solution:
    def maximumLength(self, nums: list[int]) -> int:
        n = len(nums)

        if n == 0:
            return 0
        if n == 1:
            return 1

        # Algorithm:
        # The condition (sub[i] + sub[i+1]) % 2 == R means:
        #   If R == 0: All elements in the subsequence must have the same parity (even+even=even, odd+odd=even).
        #   If R == 1: Elements in the subsequence must strictly alternate parities (even+odd=odd, odd+even=odd).

        # Case A: Subsequences where all elements have the same parity (sum % 2 == 0)
        count_even_only = 0
        count_odd_only = 0
        for num in nums:
            if num % 2 == 0:
                count_even_only += 1
            else:
                count_odd_only += 1

        # Case B: Subsequences where elements strictly alternate parities (sum % 2 == 1)

        # Subsequence starting with an even number (0, 1, 0, 1, ...)
        len_alternating_even_odd = 0
        expected_parity = 0  # Start expecting an even number
        for num in nums:
            if (num % 2) == expected_parity:
                len_alternating_even_odd += 1
                expected_parity = 1 - expected_parity  # Toggle expectation (0 -> 1, 1 -> 0)

        # Subsequence starting with an odd number (1, 0, 1, 0, ...)
        len_alternating_odd_even = 0
        expected_parity = 1  # Start expecting an odd number
        for num in nums:
            if (num % 2) == expected_parity:
                len_alternating_odd_even += 1
                expected_parity = 1 - expected_parity  # Toggle expectation (0 -> 1, 1 -> 0)

        # The maximum length is the best among all these possibilities.
        # Note: If no elements of a certain parity exist, their count/length will be 0,
        # but a single element is always a valid subsequence of length 1.
        # This is handled by initializing max_len to 1 if n >= 1.
        
        # If n=0, result is 0. If n=1, result is 1. The max() will correctly handle this
        # since count_even_only or count_odd_only will be 1, and alternating will be 1.
        return max(count_even_only, count_odd_only, len_alternating_even_odd, len_alternating_odd_even)

In [ ]:
# Approach 2: Simplified Dynamic Programming (Generalization of Approach 1, also Optimal for k=2)

class Solution:
    def maximumLength(self, nums: list[int]) -> int:
        n = len(nums)

        if n == 0:
            return 0
        
        # Algorithm:
        # This is a specialized version of the DP for "Valid Subsequence II" where k=2.
        # dp[r1][r2] stores the maximum length of a valid subsequence
        # that ends with an element whose parity is r1, and the element
        # before it in the subsequence had parity r2.
        # Parities are 0 for even, 1 for odd.
        # The structure of the subsequence is ..., r2, r1.

        # dp[0][0]: Ends with even, prev was even (sum % 2 == 0)
        # dp[0][1]: Ends with even, prev was odd (sum % 2 == 1)
        # dp[1][0]: Ends with odd, prev was even (sum % 2 == 1)
        # dp[1][1]: Ends with odd, prev was odd (sum % 2 == 0)
        
        dp = [[0] * 2 for _ in range(2)]
        
        # Initialize max_len. Any single element forms a valid subsequence of length 1.
        max_len = 1 

        for num in nums:
            current_parity = num % 2
            
            # Update dp states for the current number:
            # We iterate through all possible 'previous_parities'.
            # A new subsequence can be formed/extended that ends with 'current_parity'
            # and whose previous element (if any) had 'previous_parity'.

            for prev_parity in range(2):
                # dp[current_parity][prev_parity] represents a sequence ending with `current_parity`
                # and preceded by `prev_parity`.
                # To extend such a sequence, we look at dp[prev_parity][current_parity].
                # This refers to a sequence that ended with `prev_parity` and was preceded by `current_parity`.
                # By adding the current 'num' (which has 'current_parity'), we effectively
                # extend that previous sequence.
                # Example: If previous sequence was `..., X, Y` (X=current_parity, Y=prev_parity)
                # and now we add `num` (which has current_parity).
                # The new sequence becomes `..., X, Y, current_parity(num)`.
                # So the new pair is (current_parity, prev_parity).

                # The logic dp[current_parity][prev_parity] = dp[prev_parity][current_parity] + 1
                # correctly captures the alternating pattern:
                # If we want a subsequence `... -> p -> c`
                # We need to find the longest subsequence `... -> c -> p`
                # and append the current number.
                
                dp[current_parity][prev_parity] = max(dp[current_parity][prev_parity], dp[prev_parity][current_parity] + 1)
                
                # Each dp state represents a valid subsequence length.
                # Update the overall maximum length found.
                max_len = max(max_len, dp[current_parity][prev_parity])
        
        return max_len

In [ ]:
from typing import List

class Solution:
    def maximumLength(self, nums: List[int]) -> int:
        # Handles empty input
        if not nums:
            return 0

        k = 2  # Modulo base
        dp = [[0] * k for _ in range(k)]  # 2D table to track transitions
        max_len = 0

        for num in nums:
            x = num % k  # Current number's mod
            for j in range(k):
                # Calculate the transition target
                y = (j - x + k) % k
                # Update current DP cell based on reverse link
                dp[x][y] = dp[y][x] + 1
                max_len = max(max_len, dp[x][y])

        return max_len
def test_maximumLength():
    sol = Solution()

    # 🔹 Basic transitions
    print(sol.maximumLength([1, 2, 3, 4, 5]))           # Expected: Varies, generally ≥2

    # 🔹 All even numbers → %2 = 0
    print(sol.maximumLength([2, 4, 6, 8]))              # Expected: Higher sequence count

    # 🔹 All odd numbers → %2 = 1
    print(sol.maximumLength([1, 3, 5, 7]))              # Expected: Higher sequence count

    # 🔹 Alternating even/odd
    print(sol.maximumLength([1, 2, 3, 4]))              # Expected: 2 or more

    # 🔹 All identical numbers
    print(sol.maximumLength([9, 9, 9, 9]))              # Expected: 4

    # 🔹 Empty list
    print(sol.maximumLength([]))                       # Expected: 0

    # 🔹 Single element
    print(sol.maximumLength([42]))                     # Expected: 1

    # 🔹 Large values with modulo overlap
    print(sol.maximumLength([101, 204, 307, 408]))      # Expected: 4

    # 🔹 Stress test with 10,000 elements
    import random
    big_test = [random.randint(1, 1000) for _ in range(10000)]
    print(sol.maximumLength(big_test))                 # Should run efficiently